<a href="https://www.kaggle.com/code/taniamartynenko/eda-mll-s3e22tania?scriptVersionId=144475260" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<img src="https://www.travelwisconsin.com/uploads/medialibrary/d4/d4403f3a-1cee-4655-b47f-96afe747d93c-70-horseback-riding.jpg?preset=interior-slider-desktop" alt="Image Description"  width="1000" height="400"/>

1. [Introduction](#introduction)
2. [Import Libraries](#import_Libraries)
3. [Loading Data](#loading_Data)
4. 



<a id='introduction'></a>
## 1. Introduction

### Brief description of data set columns:


- **id**: A unique identifier for each entry. 
- **surgery**: Information on whether the horse had surgery or not. 
- **age**: Age of the horse. 
- **hospital_number**: A unique identifier for the horse in the hospital. 
- **rectal_temp**: The rectal temperature of the horse. 
- **pulse**: The pulse rate of the horse. 
- **respiratory_rate**: The rate of respiration of the horse. 
- **temp_of_extremities**: The temperature of the horse's extremities, potentially indicating circulation issues. 
- **peripheral_pulse**: Quality of the horse's peripheral pulse. 
- **mucous_membrane**: The appearance of the horse's mucous membranes. 
- **capillary_refill_time**: Time taken for color to return to an external capillary bed after pressure is applied. 
- **pain**: The horse's level of pain.
- **peristalsis**: Gut motility observation. 
- **abdominal_distention**: Severity of abdominal distention. 
- **nasogastric_tube**: Type of nasogastric tube, if present. 
- **nasogastric_reflux**: Presence and type of nasogastric reflux. 
- **nasogastric_reflux_ph**: The pH of nasogastric reflux.
- **rectal_exam_feces**: Results of the rectal examination for feces. 
- **abdomen**: General appearance of the abdomen. 
- **packed_cell_volume**: The volume percentage of red blood cells in blood. 
- **total_protein**: Total protein level in the blood. 
- **abdomo_appearance**: Appearance of the abdominal fluid.
- **abdomo_protein**: Protein content in abdominal fluid.
- **surgical_lesion**: Indicates if a surgical lesion was present. 
- **lesion_1**: Primary lesion type. 
- **lesion_2**: Secondary lesion type, if present. 
- **lesion_3**: Tertiary lesion type, if present. 
- **cp_data**: Clinical pathology data. 
- **outcome**: The health outcome of the horse. 
_________________________________________________________________________________

**Goal:** Use the above medical indicators to predict the health outcomes of horses.

<div style="background-color: #green;">

<a id='import_Libraries'></a>
<h2>2. Import Libraries</h2>

</div>



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Metrics to evaluate the model
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve,recall_score
from sklearn import tree


# For tuning the model
from sklearn.model_selection import GridSearchCV

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

<div style="background-color: #FFD580;">
<a id='#loading_Data'></a>
<h2> 3. Loading Data</h2>

</div>

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')

### Train Data:

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

Train Dataset has columns with numerical and categorical datatypes. Next will explore missinf value and check the datatype. 

### Checking dublicates and missing value: 

In [ ]:
# Let's check for duplicate values in the data
train_df.duplicated().sum()

Train data has 0 dublicate value.

In [ ]:
# Let's check for missing values in the data
round(train_df.isnull().sum() / train_df.isnull().count() * 100, 2)

**Observations:**
Traing Data has % of missing values in the next columns: 
- temp_of_extremities       3.16
- peripheral_pulse          4.86
- mucous_membrane           1.70
- capillary_refill_time     0.49
- pain                      3.56
- peristalsis               1.62
- abdominal_distention      1.86
- nasogastric_tube          6.48
- nasogastric_reflux        1.70
- rectal_exam_feces        15.38
- abdomen                  17.25
- abdomo_appearance         3.89


In [ ]:
#Checking Missing Values Pattern in Dataframe
import missingno as mn

null_cols = [i for i in train_df.columns if train_df[i].isnull().any()]
print(null_cols)
mn.matrix(train_df[null_cols])

Columns **rectal_exam_feces** has 15.38% of missing value and **abdomen** has 17.25% missing values and pattent showes that this misisng value is crossing entair dataset. Based on this observation needs to deal with this missing values for **rectal_exam_feces** and **abdomen** and do the same for test data. 

In [ ]:
# Let's check the number of unique values in each column
train_df.nunique()

In [ ]:
for i in train_df.describe(include=["object"]).columns:
    print("Unique values in", i, "are :")
    print(train_df[i].value_counts())
    print("*" * 50)

In [ ]:
train_df['lesion_1'].unique()

In [ ]:
n = 0
for value in train_df['lesion_1']:
    if value > 0:
        n += 1
print(n)


In [ ]:
train_df['lesion_2'].unique()

In [ ]:
train_df['lesion_3'].unique()

In [ ]:
train_df

In [ ]:
train_df['id'].nunique()

### Data Pre-processing - Train_df


In [ ]:
# ID column consists of uniques ID and hence will not add value to the modeling

saved_ids_train = train_df['id'].copy()

train_df.drop(columns="id", inplace=True)

In [ ]:
# ID column consists of uniques hospital_number and hence will not add value to the modeling
train_df.drop(columns="hospital_number", inplace=True)

In [ ]:
# Column lesion_1 dublicates categarical column surgical_lesion and showes type(number of) of surgery. 
#train_df.drop(columns = "lesion_1", inplace=True) 

### **Univariate analysis of numerical columns - Train_df**

In [ ]:
# Let's view the statistical summary of the numerical columns in the data
train_df.describe().T

### Univariant Analysist:

In [ ]:
train_df.dtypes

In [ ]:
num_cols = ['rectal_temp','pulse', 'respiratory_rate', 'nasogastric_reflux_ph',
            'packed_cell_volume','total_protein','abdomo_protein']
    

In [ ]:
# Set up the figure and axes
n = len(num_cols)

fig, axes = plt.subplots(n, 2, figsize=(12, 3*n))
#fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(16, 20))

# Loop through the numerical variables to plot histograms and box plots
for i, var in enumerate(num_cols):
    # Plot histogram
    axes[i, 0].hist(train_df[var], bins=30, alpha=0.7)
    axes[i, 0].set_title(f'{var} - Histogram')
    axes[i, 0].set_xlabel(var)
    axes[i, 0].set_ylabel('Frequency')
    
    # Plot box plot
    axes[i, 1].boxplot(train_df[var], vert=False)
    axes[i, 1].set_title(f'{var} - Box plot')

# Adjust the layout
plt.tight_layout()
plt.show()

### Correlation among variables Train Df

In [ ]:
correlation_matrix = train_df[num_cols].corr()
print(correlation_matrix)

### Ploting Correlation Matrix

In [ ]:
plt.figure(figsize=(8, 4))
sns.heatmap(train_df[num_cols].corr(), annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral")
plt.show()

### **Observations:**

1. **Diagonal Elements**: 
   - The diagonal elements of the correlation matrix are always 1 because any variable is perfectly correlated with itself.
   
2. **Pulse and Packed Cell Volume**:
   - There's a moderate positive correlation between `pulse` and `packed_cell_volume` (\( r \approx 0.435 \)). This suggests that as the pulse rate increases, the volume percentage of red blood cells in the blood also tends to increase.
   
3. **Pulse and Respiratory Rate**:
   - `pulse` and `respiratory_rate` also show a moderate positive correlation (\( r \approx 0.395 \)). This means that horses with a higher pulse rate tend to have a higher respiratory rate.

4. **Nasogastric Reflux pH and Total Protein**:
   - There's a strong negative correlation between `nasogastric_reflux_ph` and `total_protein` (\( r \approx -0.583 \)). As the pH of nasogastric reflux increases, the total protein level in the blood tends to decrease.

5. **Nasogastric Reflux pH and Abdomo Protein**:
   - There's a positive correlation between `nasogastric_reflux_ph` and `abdomo_protein` (\( r \approx 0.433 \)). This means that as the pH of nasogastric reflux increases, the protein content in abdominal fluid also tends to increase.

6. **Lesion 2 and Lesion 3**:
   - There's a strong positive correlation between `lesion_2` and `lesion_3` (\( r \approx 0.644 \)). This suggests that the secondary and tertiary lesion types tend to occur together.

7. **Weak Correlations**:
   - Many of the correlations in the matrix are close to zero, indicating weak or no linear relationship between the paired variables.



## Handling missing values in train_df:

### -  Columns with Numeric Data (float64/int64):

For columns like **rectal_temp, pulse, respiratory_rate, nasogastric_reflux_ph, packed_cell_volume, total_protein, and abdomo_protein,**  can be use the median or mean of the column to fill in missing values.

In [ ]:
train_df['rectal_temp'].fillna(train_df['rectal_temp'].median(), inplace=True)
train_df['pulse'].fillna(train_df['pulse'].median(), inplace=True)
train_df['respiratory_rate'].fillna(train_df['respiratory_rate'].median(), inplace=True)
train_df['nasogastric_reflux_ph'].fillna(train_df['nasogastric_reflux_ph'].median(), inplace=True)
train_df['packed_cell_volume'].fillna(train_df['packed_cell_volume'].median(), inplace=True)
train_df['total_protein'].fillna(train_df['total_protein'].median(), inplace=True)
train_df['abdomo_protein'].fillna(train_df['abdomo_protein'].median(), inplace=True)


### -  Columns with Categorical Data (Object):

In [ ]:
train_df['rectal_exam_feces'].fillna('Unknown', inplace=True)
train_df['abdomen'].fillna('Unknown', inplace=True)

temp_of_extremities,
peripheral_pulse, 
mucous_membrane, 
capillary_refill_time,  
pain, 
peristalsis, 
abdominal_distention,  
nasogastric_tube,  
 nasogastric_reflux,  
 abdomo_appearance 
 

In [ ]:
train_df['temp_of_extremities'].fillna(train_df['temp_of_extremities'].mode()[0], inplace=True)
train_df['peripheral_pulse'].fillna('Unknown', inplace=True)

train_df['mucous_membrane'].fillna(train_df['mucous_membrane'].mode()[0], inplace=True)
train_df['capillary_refill_time'].fillna('Unknown', inplace=True)

train_df['capillary_refill_time'].fillna(train_df['capillary_refill_time'].mode()[0], inplace=True)
train_df['pain'].fillna('Unknown', inplace=True)

train_df['peristalsis'].fillna(train_df['peristalsis'].mode()[0], inplace=True)
train_df['abdominal_distention'].fillna('Unknown', inplace=True)

train_df['nasogastric_tube'].fillna(train_df['nasogastric_tube'].mode()[0], inplace=True)
train_df['nasogastric_reflux'].fillna('Unknown', inplace=True)

train_df['nasogastric_reflux'].fillna(train_df['nasogastric_reflux'].mode()[0], inplace=True)
train_df['abdomo_appearance'].fillna('Unknown', inplace=True)



In [ ]:
print(train_df.isnull().sum())

## Test Data

In [ ]:
test_df.shape

Test datset has 824 rows and 28 columns.

In [ ]:
test_df.head()

In [ ]:
test_df.info()

Test Dataset has columns with numerical and categorical datatypes. Next will explore missinf value and check the datatype. 

### Checking dublicates and missing value: 

In [ ]:
# Let's check for duplicate values in the data
test_df.duplicated().sum()

Test Data has 0 dublicates.

In [ ]:
# Let's check for missing values in the data
round(test_df.isnull().sum() / test_df.isnull().count() * 100, 2)

Test data has columns with missing value, %: 
- temp_of_extremities       4.25
- peripheral_pulse          5.70
- mucous_membrane           1.58
- capillary_refill_time     0.73
- pain                      3.52
- peristalsis               2.31
- abdominal_distention      2.67
- nasogastric_tube          7.77
- nasogastric_reflux        1.70
- rectal_exam_feces        15.17
- abdomen                  18.69
- abdomo_appearance         3.76

In [ ]:
#Checking Missing Values Pattern in Dataframe
import missingno as mn

null_cols = [i for i in test_df.columns if test_df[i].isnull().any()]
print(null_cols)
mn.matrix(test_df[null_cols])

In [ ]:
# Let's check the number of unique values in each column
test_df.nunique()

#### Data Pre-processing Test_df

In [ ]:
# ID column consists of uniques ID and hence will not add value to the modeling

saved_ids_test = test_df['id'].copy()

test_df.drop(columns="id", inplace=True)

In [ ]:
# ID column consists of uniques hospital_number and hence will not add value to the modeling
test_df.drop(columns="hospital_number", inplace=True)

 ### Impute Missing Values in Test Dataset:

#### - Numeric Columns (float64/int64):

In [ ]:
# Using the median of the training dataset to fill missing values in the test dataset
test_df['rectal_temp'].fillna(train_df['rectal_temp'].median(), inplace=True)
test_df['pulse'].fillna(train_df['pulse'].median(), inplace=True)
test_df['respiratory_rate'].fillna(train_df['respiratory_rate'].median(), inplace=True)
test_df['nasogastric_reflux_ph'].fillna(train_df['nasogastric_reflux_ph'].median(), inplace=True)
test_df['packed_cell_volume'].fillna(train_df['packed_cell_volume'].median(), inplace=True)
test_df['total_protein'].fillna(train_df['total_protein'].median(), inplace=True)
test_df['abdomo_protein'].fillna(train_df['abdomo_protein'].median(), inplace=True)


#### -Categorical Columns (object):

In [ ]:
# Using the mode of the training dataset or 'Unknown' to fill missing values in the test dataset
test_df['temp_of_extremities'].fillna(train_df['temp_of_extremities'].mode()[0], inplace=True)
test_df['peripheral_pulse'].fillna('Unknown', inplace=True)
test_df['mucous_membrane'].fillna(train_df['mucous_membrane'].mode()[0], inplace=True)
test_df['capillary_refill_time'].fillna(train_df['capillary_refill_time'].mode()[0], inplace=True)
test_df['pain'].fillna(train_df['pain'].mode()[0], inplace=True)
test_df['peristalsis'].fillna(train_df['peristalsis'].mode()[0], inplace=True)
test_df['abdominal_distention'].fillna(train_df['abdominal_distention'].mode()[0], inplace=True)
test_df['nasogastric_tube'].fillna(train_df['nasogastric_tube'].mode()[0], inplace=True)
test_df['nasogastric_reflux'].fillna(train_df['nasogastric_reflux'].mode()[0], inplace=True)
test_df['abdomo_appearance'].fillna(train_df['abdomo_appearance'].mode()[0], inplace=True)


#### - Special Columns:

In [ ]:
# Using 'Unknown' to fill missing values for specific columns
test_df['rectal_exam_feces'].fillna('Unknown', inplace=True)
test_df['abdomen'].fillna('Unknown', inplace=True)


In [ ]:
print(test_df.isnull().sum())


In [ ]:
train_df

### Encoding Categorical Columns

* We will be using the Label Encoding technique to encode the values of the categorical columns in this dataset.

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_RE = LabelEncoder()
train_df['surgical_lesion']= labelencoder_RE.fit_transform(train_df['surgical_lesion'])
test_df['surgical_lesion']= labelencoder_RE.transform(test_df['surgical_lesion'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['cp_data'] =  labelencoder_EN.fit_transform(train_df['cp_data'])
test_df['cp_data'] =  labelencoder_EN.transform(test_df['cp_data'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['surgical_lesion'] =  labelencoder_EN.fit_transform(train_df['surgical_lesion'])
test_df['surgical_lesion'] =  labelencoder_EN.transform(test_df['surgical_lesion'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['rectal_exam_feces'] =  labelencoder_EN.fit_transform(train_df['rectal_exam_feces'])
test_df['rectal_exam_feces'] =  labelencoder_EN.transform(test_df['rectal_exam_feces'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['abdomen'] =  labelencoder_EN.fit_transform(train_df['abdomen'])
test_df['abdomen'] =  labelencoder_EN.transform(test_df['abdomen'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['abdomo_appearance'] =  labelencoder_EN.fit_transform(train_df['abdomo_appearance'])
test_df['abdomo_appearance'] =  labelencoder_EN.transform(test_df['abdomo_appearance'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['age'] =  labelencoder_EN.fit_transform(train_df['age'])
test_df['age'] =  labelencoder_EN.transform(test_df['age'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['surgery'] =  labelencoder_EN.fit_transform(train_df['surgery'])
test_df['surgery'] =  labelencoder_EN.transform(test_df['surgery'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['temp_of_extremities'] =  labelencoder_EN.fit_transform(train_df['temp_of_extremities'])
test_df['temp_of_extremities'] =  labelencoder_EN.transform(test_df['temp_of_extremities'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['peripheral_pulse'] =  labelencoder_EN.fit_transform(train_df['peripheral_pulse'])
test_df['peripheral_pulse'] =  labelencoder_EN.transform(test_df['peripheral_pulse'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['mucous_membrane'] =  labelencoder_EN.fit_transform(train_df['mucous_membrane'])
test_df['mucous_membrane'] =  labelencoder_EN.transform(test_df['mucous_membrane'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['capillary_refill_time'] =  labelencoder_EN.fit_transform(train_df['capillary_refill_time'])
test_df['capillary_refill_time'] =  labelencoder_EN.transform(test_df['capillary_refill_time'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['pain'] =  labelencoder_EN.fit_transform(train_df['pain'])

In [ ]:
# Fit the LabelEncoder on all unique values from both DataFrames
labelencoder_EN = LabelEncoder().fit(test_df['pain'].unique())

test_df['pain'] = labelencoder_EN.transform(test_df['pain'])


In [ ]:
labelencoder_EN = LabelEncoder()
train_df['peristalsis'] =  labelencoder_EN.fit_transform(train_df['peristalsis'])
test_df['peristalsis'] =  labelencoder_EN.transform(test_df['peristalsis'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['abdominal_distention'] =  labelencoder_EN.fit_transform(train_df['abdominal_distention'])
test_df['abdominal_distention'] =  labelencoder_EN.transform(test_df['abdominal_distention'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['nasogastric_tube'] =  labelencoder_EN.fit_transform(train_df['nasogastric_tube'])
test_df['nasogastric_tube'] =  labelencoder_EN.transform(test_df['nasogastric_tube'])

In [ ]:
labelencoder_EN = LabelEncoder()
train_df['nasogastric_reflux'] =  labelencoder_EN.fit_transform(train_df['nasogastric_reflux'])
test_df['nasogastric_reflux'] =  labelencoder_EN.transform(test_df['nasogastric_reflux'])


In [ ]:
train_df.info()

In [ ]:
test_df.info()

### Modeling

#### Data Preparation

In [ ]:
# Load the sample submission file
sample_submission_filepath = "/kaggle/input/playground-series-s3e22/sample_submission.csv"
sample_submission = pd.read_csv(sample_submission_filepath)

# Update the target column with your predictions
#sample_submission['target_column_name'] = y_pred_test

# Save the submission file
#sample_submission.to_csv('submission.csv', index=False)

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.isnull().sum()

No missing values.

In [ ]:
sample_submission.nunique()

In [ ]:
sample_submission.info()

In [ ]:
# Define the mappings
mappings = {
    'outcome': {'lived': 1, 'died': 0, 'euthanized': 0}
}

# Apply the mappings to the dataframe
for column, mapping in mappings.items():
    sample_submission[column] = sample_submission[column].map(mapping)

In [ ]:
mappings = {
    'outcome': {'lived': 1, 'died': 0, 'euthanized': 0}
}

# Apply the mappings to the dataframe
for column, mapping in mappings.items():
    train_df['outcome'] = train_df['outcome'].map(mapping)

### Scaling Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train = train_df.drop(columns='outcome')
y_train = train_df['outcome']#.astype(int)

#X_train, y_train = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
y_train = pd.DataFrame(y_train, columns=['outcome'])

### Scalling Data: 

In [ ]:
# Scaling the data
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X_train.columns)



In [ ]:
X_test_scaled = sc.transform(test_df)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=test_df.columns)

In [ ]:
y_test = sample_submission['outcome']

### Model evaluation criterion

**Goal: Use the above medical indicators to predict the health outcomes of horses.**  

Given the goal of predicting the health outcomes of horses, the two types of wrong predictions your model can make are:

1. **False Positive (FP)**: Predicting that a horse will experience a certain health outcome when it will not. For example, predicting that a horse will "die" when it actually "lives."
2. **False Negative (FN)**: Predicting that a horse will not experience a certain health outcome when it will. For example, predicting that a horse will "live" when it actually "dies."

The implications of these errors in the context of predicting health outcomes for horses are:

- A **False Positive** might lead to unnecessary interventions, treatments, or even euthanasia if the prediction is severe. It can also lead to unnecessary costs and stress.
  
- A **False Negative** can be more dangerous. If the model predicts that a horse is going to be fine ("live") when it's actually at risk ("die" or "euthanized"), it might not receive the necessary medical attention in time, leading to potential harm or death.

In many medical or health-related contexts, reducing **False Negatives** is often more critical because of the potential harm that can arise from missing a crucial diagnosis or intervention. 

- Evaluation metric should give more penalty to the type of error you deem more critical.  Given that using the **F1-Score**, it takes both precision and recall into account, balancing the trade-off between **False Positives** and **False Negatives**.

In [ ]:
# Function to print the classification report and get confusion matrix in a proper format

def metrics_score(actual, predicted):
    print(classification_report(actual, predicted))
    
    cm = confusion_matrix(actual, predicted)
    
    plt.figure(figsize = (8, 5))
    
    sns.heatmap(cm, annot = True,  fmt = '.2f', xticklabels = ['0', '1'], yticklabels = ['0', '1'])
    
    plt.ylabel('Actual')
    
    plt.xlabel('Predicted')
    
    plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def metrics_score1(actual, predicted):
    # Print the classification report
    print(classification_report(actual, predicted))
    
    # Compute the confusion matrix
    cm = confusion_matrix(actual, predicted)
    
    # Get unique class labels
    class_labels = sorted(list(set(actual) | set(predicted)))
    
    plt.figure(figsize=(8, 5))
    
    # Plot the heatmap
    sns.heatmap(cm, annot=True, fmt='.2f', xticklabels=class_labels, yticklabels=class_labels)
    
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()


## Building a Decision Tree model

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_scaled, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_dt = dt_classifier.predict(X_train_scaled)
score = metrics_score(y_train, y_train_pred_dt)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = dt_classifier.predict(X_test_scaled)
score = metrics_score(y_test, y_test_pred_dt)

In [ ]:
# Importance of features in the tree building

Imp_f_d_tree = pd.DataFrame(dt_classifier.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_d_tree 

In [ ]:
# Plot the feature importance

importances = dt_classifier.feature_importances_
columns = X_train.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

### Visualizing the Decision Tree

In [ ]:
features = list(X_train.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(dt_classifier, max_depth = 6, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

### Prune Tree Model

In [ ]:
# Get unique classes in y_train
unique_classes = np.unique(y_train)

# If you have specific weights you want to assign, you can do so here
class_weights = {cls: weight for cls, weight in zip(unique_classes, [0.3, 0.7, 0.7])}

# Update the class_weight parameter in the classifier
d_tree_tuned = DecisionTreeClassifier(random_state=7, class_weight=class_weights)

# Continue with the rest of your code
parameters = {'max_depth': np.arange(2, 10), 
              'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [5, 10, 20, 25]
             }

scorer = metrics.make_scorer(recall_score, pos_label=1)

grid_obj = GridSearchCV(d_tree_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

d_tree_tuned = grid_obj.best_estimator_
d_tree_tuned.fit(X_train_scaled, y_train)


In [ ]:
# Checking performance on the training data

y_train_pred_dt2 = d_tree_tuned.predict(X_train_scaled)

score_train = metrics_score(y_train, y_train_pred_dt2)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = d_tree_tuned.predict(X_test_scaled)
score = metrics_score(y_test, y_test_pred_dt)

#### Let's visualize the tuned decision tree and observe the decision rules:

In [ ]:
features = list(X_train.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(d_tree_tuned, max_depth = 6, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

In [ ]:
# Importance of features in the tree building

Imp_f_d_tuned = pd.DataFrame(d_tree_tuned.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_d_tuned

In [ ]:
# Plot the feature importance

importances = d_tree_tuned.feature_importances_
columns = X_train.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

### Building a Random Forest model

In [ ]:
#Create an instance of the RandomForestClassifier
rf_estimator = RandomForestClassifier(random_state=42, n_estimators=100)

#Fit the model on the training data
rf_estimator.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions on the test data
y_test_pred_rf = rf_estimator.predict(X_test_scaled)

In [ ]:
#Predicting the training data labels
y_pred_train3 = rf_estimator.predict(X_train_scaled)

score_train_rf = metrics_score(y_train, y_pred_train3)

In [ ]:
#Predicting the test data labels
y_test_rf3 = rf_estimator.predict(X_test_scaled)

score_test_rf = metrics_score(y_test, y_test_rf3)

### Random Forest Classifier - Hyperparameter Tuning

In [ ]:
# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(criterion = "entropy", random_state = 7)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": ["balanced", {0: 0.3, 1: 0.7}]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search on the training data
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

# Display the best parameters found by the grid search
print('Best parameters found: ', grid_obj.best_params_)

In [ ]:
rf_estimator_tuned.fit(X_train_scaled, y_train)

In [ ]:
# Checking performance on the training data

y_pred_train_rft = rf_estimator_tuned.predict(X_train_scaled)

score_train_rft = metrics_score(y_train, y_pred_train_rft)

In [ ]:
# Checking performance on the test data

y_pred_test_rft = rf_estimator_tuned.predict(X_test_scaled)

score_test_rft = metrics_score(y_test, y_pred_test_rft)

In [ ]:
# Importance of features in the tree building

Imp_f_fr_tuned = pd.DataFrame(rf_estimator_tuned.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_fr_tuned 

In [ ]:
importances = rf_estimator_tuned.feature_importances_ 
columns = X_train.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

### LightGBM (Light Gradient Boosting Machine)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
# Instantiate the LGBMClassifier with 100 estimators (trees) and a random state for reproducibility
lgbm_md = LGBMClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data (df_train), excluding the target variable
lgbm_md.fit(X_train_scaled, y_train)

In [ ]:
# Use the trained model to make predictions on the test data (df_test)
y_lgbm_pred = lgbm_md.predict(X_test_scaled)

In [ ]:
# Checking performance on the train data

y_lgbm_pred_train = lgbm_md.predict(X_train_scaled)

score_test_lgbm = metrics_score(y_train, y_lgbm_pred_train)

In [ ]:
# Checking performance on the test data

y_lgbm_pred_test = lgbm_md.predict(X_test_scaled)

score_test_lgbm = metrics_score(y_test, y_lgbm_pred_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.metrics import recall_score

# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(criterion="entropy", random_state=7)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": ["balanced", {0: 0.3, 1: 0.7}]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label=1)

# Run the grid search on the training data
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

# Display the best parameters found by the grid search
print('Best parameters found: ', grid_obj.best_params_)

# Checking performance on the training data
y_pred_train_rft = rf_estimator_tuned.predict(X_train_scaled)
score_train_rft = metrics_score(y_train, y_pred_train_rft)

# Define the number of folds
k = 5

# Perform k-fold cross-validation
scores = cross_val_score(rf_estimator_tuned, X_train_scaled, y_train, cv=k, scoring='f1_macro')

# Print the scores for each fold
print(f'F1 Score for each fold: {scores}')

# Print the average score
print(f'Average F1 Score: {scores.mean()}')



In [ ]:
# Assuming y_test is your true labels for the test set
y_pred_test_rft = rf_estimator_tuned.predict(X_test_scaled)
score_test_rft = metrics_score(y_test, y_pred_test_rft)

# Print the F1 Score for the test set
f1_score_test = metrics.f1_score(y_test, y_pred_test_rft, average='macro')
print(f'F1 Score for the test set: {f1_score_test}')


### Build the Neural Network model:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score


In [ ]:
# Define the model
model = Sequential()

# Add input layer (the input dimension is the number of features)
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))

# Add dropout layer for regularization
model.add(Dropout(0.5))

# Add hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Add output layer (the output dimension is the number of classes)
model.add(Dense(3, activation='softmax'))  # Assuming you have 3 classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


In [ ]:
# Adjust the labels in the training and test sets
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1


In [ ]:
# Fit the model to the adjusted training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the adjusted test data
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Make predictions on the test set
y_test_pred_proba = model.predict(X_test_scaled)

# Convert predicted probabilities to class labels
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

# Adjust the predictions back to the original label range
y_test_pred_adjusted = y_test_pred

# Calculate the F1 Score
f1 = f1_score(y_test, y_test_pred_adjusted, average='macro')
print("F1 Score on Test Set:", f1)


In [ ]:
print("Number of negative labels:", (y_train_adjusted < 0).sum())

In [ ]:
# Fixing the seed for random number generators
np.random.seed(42)

import random
random.seed(42)

tf.random.set_seed(42)

In [ ]:
print(X_train_scaled.shape)

In [ ]:
# We will be adding the layers sequentially
model_1 = Sequential()

# First hidden layer with 128 neurons and relu activation function, the input shape tuple denotes number of independent variables
model_1.add(Dense(128, activation='relu', input_shape=(26,)))

# We will be switching 20% of neurons off randomly at each iteration to avoid overfitting
model_1.add(Dropout(0.2))

# Second hidden layer with 64 neurons and relu activation function
model_1.add(Dense(64, activation='relu'))

# We will be switching 10% of neurons off randomly at each iteration to avoid overfitting
model_1.add(Dropout(0.1))

# Output layer with only one neuron and sigmoid as activation function will give the probability of students getting admitted into UCLA  
model_1.add(Dense(1, activation='sigmoid'))

In [ ]:
model_1.compile(loss = 'binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

model_1.summary()

In [ ]:
history_1 = model_1.fit(X_train_scaled, 
                    y_train,
                    validation_split=0.1, 
                    epochs=150, 
                    verbose=2)

In [ ]:
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.title('Accuracy vs Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model_1.evaluate(X_test_scaled, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# Clearing the backend
from tensorflow.keras import backend
backend.clear_session()

In [ ]:
# Fixing the seed for random number generators
np.random.seed(42)

import random
random.seed(42)

tf.random.set_seed(42)

In [ ]:
model_3 = Sequential()
model_3.add(Dense(256, activation='tanh', input_shape=(26,)))
model_3.add(Dropout(0.2))
model_3.add(Dense(128, activation='tanh'))
model_3.add(Dropout(0.2))
model_3.add(Dense(64, activation='tanh'))
model_3.add(Dropout(0.2))
model_3.add(Dense(32, activation='tanh'))
model_3.add(Dense(1, activation='sigmoid'))

In [ ]:
model_3.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model_3.summary()

In [ ]:
history_3 = model_3.fit(X_train_scaled, 
                    y_train, 
                    validation_split=0.2, 
                    epochs=150,
                    verbose=2)

In [ ]:
model_3.evaluate(X_test_scaled, y_test, verbose = 1)
test_pred = np.round(model_3.predict(X_test_scaled))

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model_3.evaluate(X_test_scaled, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('Accuracy vs Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(classification_report(y_test, test_pred))
cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(8,5))
sns.heatmap(cm, annot=True,  fmt='.0f',xticklabels=['Died', 'Lived'], yticklabels=['Died', 'Lived'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

# Here, we are passing all the features (numerical and categorical), that's okay as min-max scaler will not change values of categorical variables
X_train_normalized = scaler.fit_transform(X_train)

In [ ]:
X_test_normalized =scaler.fit_transform(test_df)

In [ ]:
history_3 = model_3.fit(X_train_normalized, 
                    y_train, 
                    validation_split=0.2, 
                    epochs=150,
                    verbose=2)

In [ ]:
model_3.evaluate(X_test_normalized, y_test, verbose = 1)
test_pred = np.round(model_3.predict(X_test_normalized))

### PCA

In [ ]:
X_train_scaled

Based on resuld from DT future importance lets take PCA for bottom futures with lowest importance: 'peristalsis', 'respiratory_rate', 'nasogastric_reflux', 'age', 'capillary_refill_time', 'cp_data', 'lesion_2', 'surgical_lesion', 'lesion_3'

In [ ]:
X_train_scaled. info()

In [ ]:
from sklearn.decomposition import PCA

columns_for_pca = ['peristalsis', 'respiratory_rate', 'nasogastric_reflux', 'age', 'capillary_refill_time', 'cp_data', 'lesion_2', 'surgical_lesion', 'lesion_3']
X_train_selected = X_train_scaled[columns_for_pca]

pca = PCA(n_components=2)

# Fit the PCA 
X_train_pca = pca.fit_transform(X_train_selected)

X_train_scaled['Principal Component 1'] = X_train_pca[:, 0]

X_train_scaled['Principal Component 2'] = X_train_pca[:, 1]

X_train_scaled.drop(columns=columns_for_pca, inplace=True)

In [ ]:
X_train_scaled.head()

Aply PCA on test data: 

In [ ]:
X_test_pca = pca.transform(X_test_scaled[columns_for_pca])


X_test_scaled['Principal Component 1'] = X_test_pca[:, 0]
X_test_scaled['Principal Component 2'] = X_test_pca[:, 1]

# Create a new DataFrame with the specified columns dropped
X_test_scaled.drop(columns=columns_for_pca, inplace=True)


In [ ]:
X_test_scaled.head()

## Biulding ML with data PCA

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_scaled, y_train)

In [ ]:
# Checking performance on the training dataset
y_train_pred_dt = dt_classifier.predict(X_train_scaled)
score = metrics_score(y_train, y_train_pred_dt)

In [ ]:
# Checking performance on the test dataset
y_test_pred_dt = dt_classifier.predict(X_test_scaled)
score = metrics_score(y_test, y_test_pred_dt)

In [ ]:
# Importance of features in the tree building

Imp_f_d_tree = pd.DataFrame(dt_classifier.feature_importances_, columns = ["Imp"], index = X_train_scaled.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_d_tree 

In [ ]:
# Plot the feature importance

importances = dt_classifier.feature_importances_
columns = X_train_scaled.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

In [ ]:
features = list(X_train.columns)

plt.figure(figsize = (30, 20))

tree.plot_tree(dt_classifier, max_depth = 6, feature_names = features, filled = True, fontsize = 12, node_ids = True, class_names = True)

plt.show()

### Prune Tree Model with Data with PCA

In [ ]:
# Get unique classes in y_train
unique_classes = np.unique(y_train)

# If you have specific weights you want to assign, you can do so here
class_weights = {cls: weight for cls, weight in zip(unique_classes, [0.3, 0.7, 0.7])}

# Update the class_weight parameter in the classifier
d_tree_tuned = DecisionTreeClassifier(random_state=7, class_weight=class_weights)

# Continue with the rest of your code
parameters = {'max_depth': np.arange(2, 10), 
              'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [5, 10, 20, 25]
             }

scorer = metrics.make_scorer(recall_score, pos_label=1)

grid_obj = GridSearchCV(d_tree_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

d_tree_tuned = grid_obj.best_estimator_
d_tree_tuned.fit(X_train_scaled, y_train)

In [ ]:

# Define the parameter grid
param_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': [0.0, 0.01, 0.1, 1.0]
}

# Initialize the DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit the model on the training data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)


In [ ]:

# Initialize the DecisionTreeClassifier with the best parameters
clf = DecisionTreeClassifier(ccp_alpha=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=2, random_state=42)

# Fit the model on the training data
clf.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test_scaled)

# Evaluate the model accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Initialize the DecisionTreeClassifier with the best parameters
clf = DecisionTreeClassifier(ccp_alpha=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=2, random_state=42)

# Perform k-fold cross-validation (e.g., 5-fold)
cv_scores = cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring='accuracy')

# Calculate the mean and standard deviation of the cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

print("Mean Cross-Validation Score:", mean_cv_score)
print("Standard Deviation of Cross-Validation Score:", std_cv_score)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Fit the classifier with the best parameters on the entire training dataset
clf.fit(X_train_scaled, y_train)

# Predict the labels for the test data
y_test_pred = clf.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_test_pred, average='weighted')
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_test_pred, average='weighted')
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(y_test, y_test_pred, average='weighted')
print("F1 Score:", f1)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", cm)


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print("Class Distribution after Oversampling:")
print(y_train_resampled.value_counts())

In [ ]:
# Checking performance on the training data

y_train_pred_dt2 = d_tree_tuned.predict(X_train_scaled)

score_train = metrics_score(y_train, y_train_pred_dt2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Initialize the DecisionTreeClassifier with the best parameters
clf = DecisionTreeClassifier(ccp_alpha=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=2, random_state=42)

# Fit the model on the balanced training data
clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_test_pred = clf.predict(X_test_scaled)
print("Test Data perfomance: ")
# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, y_test_pred, average='weighted')
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_test, y_test_pred, average='weighted')
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(y_test, y_test_pred, average='weighted')
print("F1 Score:", f1)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", cm)


### Random Forest for PCA

In [ ]:
# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(criterion = "entropy", random_state = 7)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": ["balanced", {0: 0.3, 1: 0.7}]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search on the training data
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

# Display the best parameters found by the grid search
print('Best parameters found: ', grid_obj.best_params_)
rf_estimator_tuned.fit(X_train_scaled, y_train)
# Checking performance on the training data

y_pred_train_rft = rf_estimator_tuned.predict(X_train_scaled)

score_train_rft = metrics_score(y_train, y_pred_train_rft)


In [ ]:
# Checking performance on the test data

y_pred_test_rft = rf_estimator_tuned.predict(X_test_scaled)

score_test_rft = metrics_score(y_test, y_pred_test_rft)

In [ ]:
# Importance of features in the tree building

Imp_f_fr_tuned = pd.DataFrame(rf_estimator_tuned.feature_importances_, columns = ["Imp"], index = X_train_scaled.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_fr_tuned 
importances = rf_estimator_tuned.feature_importances_ 
columns = X_train_scaled.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

In [ ]:
Imp_f_fr_tuned 

abdomen	0.007952
surgery	0.007158
mucous_membrane	0.006916
peripheral_pulse	0.005146
nasogastric_tube	0.003809

In [ ]:
columns_to_drop = ['abdomen', 'surgery', 'mucous_membrane', 'peripheral_pulse', 'nasogastric_tube']

In [ ]:
X_train_scaled.drop(columns = columns_to_drop,  inplace=True)

In [ ]:
X_test_scaled.drop(columns = columns_to_drop,  inplace=True)

In [ ]:
# Choose the type of classifier
rf_estimator_tuned = RandomForestClassifier(criterion = "entropy", random_state = 7)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [110, 120],
    "max_depth": [6, 7],
    "min_samples_leaf": [20, 25],
    "max_features": [0.8, 0.9],
    "max_samples": [0.9, 1],
    "class_weight": ["balanced", {0: 0.3, 1: 0.7}]
}

# Type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(recall_score, pos_label = 1)

# Run the grid search on the training data
grid_obj = GridSearchCV(rf_estimator_tuned, parameters, scoring=scorer, cv=5)
grid_obj = grid_obj.fit(X_train_scaled, y_train)

# Save the best estimator to variable rf_estimator_tuned
rf_estimator_tuned = grid_obj.best_estimator_

# Display the best parameters found by the grid search
print('Best parameters found: ', grid_obj.best_params_)
rf_estimator_tuned.fit(X_train_scaled, y_train)
# Checking performance on the training data

y_pred_train_rft = rf_estimator_tuned.predict(X_train_scaled)

score_train_rft = metrics_score(y_train, y_pred_train_rft)

In [ ]:
# Checking performance on the test data

y_pred_test_rft = rf_estimator_tuned.predict(X_test_scaled)

score_test_rft = metrics_score(y_test, y_pred_test_rft)

In [ ]:
# Importance of features in the tree building

Imp_f_fr_tuned = pd.DataFrame(rf_estimator_tuned.feature_importances_, columns = ["Imp"], index = X_train_scaled.columns).sort_values(by = 'Imp', ascending = False)
Imp_f_fr_tuned 
importances = rf_estimator_tuned.feature_importances_ 
columns = X_train_scaled.columns
importance_df = pd.DataFrame(importances, index = columns, columns = ['Importance']).sort_values(by = 'Importance', ascending = False)
plt.figure(figsize = (8, 8))
sns.barplot(data = importance_df, x = importance_df.Importance, y = importance_df.index);

## NN

In [ ]:
# Define the model
model = Sequential()

# Add input layer (the input dimension is the number of features)
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))

# Add dropout layer for regularization
model.add(Dropout(0.5))

# Add hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Add output layer (the output dimension is the number of classes)
model.add(Dense(3, activation='softmax'))  # Assuming you have 3 classes

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
# Adjust the labels in the training and test sets
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1
# Fit the model to the adjusted training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the adjusted test data
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


In [ ]:

# Make predictions on the test set
y_test_pred_proba = model.predict(X_test_scaled)

# Convert predicted probabilities to class labels
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

# Adjust the predictions back to the original label range
y_test_pred_adjusted = y_test_pred

# Calculate the F1 Score
f1 = f1_score(y_test, y_test_pred_adjusted, average='macro')
print("F1 Score on Test Set:", f1)

In [ ]:
X_train_scaled.info()

In [ ]:
# Clearing the backend
from tensorflow.keras import backend
backend.clear_session()
# Fixing the seed for random number generators
np.random.seed(42)

import random
random.seed(42)

tf.random.set_seed(42)
model_3 = Sequential()
model_3.add(Dense(256, activation='tanh', input_shape=(14,)))
model_3.add(Dropout(0.2))
model_3.add(Dense(128, activation='tanh'))
model_3.add(Dropout(0.2))
model_3.add(Dense(64, activation='tanh'))
model_3.add(Dropout(0.2))
model_3.add(Dense(32, activation='tanh'))
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model_3.summary()
history_3 = model_3.fit(X_train_scaled, 
                    y_train, 
                    validation_split=0.2, 
                    epochs=150,
                    verbose=2)
model_3.evaluate(X_test_scaled, y_test, verbose = 1)
test_pred = np.round(model_3.predict(X_test_scaled))
# Evaluate the model on the test set
test_loss, test_accuracy = model_3.evaluate(X_test_scaled, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('Accuracy vs Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:

print(classification_report(y_test, test_pred))
cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(8,5))
sns.heatmap(cm, annot=True,  fmt='.0f',xticklabels=['Not Admitted', 'Admitted'], yticklabels=['Not Admitted', 'Admitted'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()